In [460]:
#Import Dependencies
import pandas as pd
from pathlib import Path
import random
from datetime import datetime

In [461]:
customer_shopping_data_csv = Path("./Resources/source/original/customer_shopping_data.csv")

customer_shopping_data_df = pd.read_csv(customer_shopping_data_csv)
customer_shopping_data_df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon


In [462]:
exchange_rate_csv = Path("./Resources/output/exchange_rate.csv")

exchange_rate_df = pd.read_csv(exchange_rate_csv)

exchange_rate_df.set_index("date", inplace=True)

exchange_rate_df.head()

,exchange_rate
date,
2021-01-01,7.439200
2021-01-02,7.439200
2021-01-03,7.441906
2021-01-04,7.427000
2021-01-05,7.390200


In [463]:
config_dict = [
    {
        "category": "Books",    
        "price": {              #configuration for calculating the price
            "price_segments": [("budget", 5), ("medium", 2), ("premium", 1)],  #probability of segment selection
            "price_range": {
                "budget": {                 
                    "Female": (0.5, 3),     #format: (minimum price, maximum price)
                    "Male": (0.5, 2)
                },
                "medium": {
                    "Female": (3, 5),
                    "Male": (2, 4)
                },
                "premium": {
                    "Female": (5, 10),
                    "Male": (4, 10)
                }
            },
        },
        #distribution of prices by months. Format: (month, price coefficient)
        "price_month_coefficients": [(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, 1.1), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 0.95), (12, 0.85)],
        "quantity": {       #configuration for calculating the quantity. Format: (number of items in transaction, weight - a relative indicator of the probability of such a value)
            "Female": [(1, 3), (2, 1), (3, 0.5)], 
            "Male": [(1, 3), (2, 1.5), (3, 0.5), (4, 0.15), (5, 0.075)]
        },
        #distribution of quantity by months. Format: (month, transaction quantity coefficient)
        "quantity_month_coefficients": [(1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.15)]
    },
    {
        "category": "Clothing",
        "price": {
            "price_segments": [("budget", 5), ("medium", 2), ("premium", 1)],
            "price_range": {
                "budget": {
                    "Female": (10, 50),
                    "Male": (5, 25)
                },
                "medium": {
                    "Female": (50, 200),
                    "Male": (25, 100)
                },
                "premium": {
                    "Female": (200, 1000),
                    "Male": (100, 750)
                }
            }
        },
        "price_month_coefficients": [(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, 1.1), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 0.95), (12, 0.85)],
        "quantity": {
            "Female": [(1, 2), (2, 3), (3, 2), (4, 1), (5, 0.5), (6, 0.25), (7, 0.15), (8, 0.1), (9, 0.05), (10, 0.025)],
            "Male": [(1, 3), (2, 1.5), (3, 0.5), (4, 0.25), (5, 0.15), (6, 0.05), (7, 0.025)]
        },
        "quantity_month_coefficients": [(1, 1.25), (2, 0.75), (3, 1.15), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.5)]
    },
    {
        "category": "Cosmetics",
        "price": {
            "price_segments": [("budget", 5), ("medium", 2), ("premium", 1)],
            "price_range": {
                "budget": {
                    "Female": (5, 20),
                    "Male": (3, 10)
                },
                "medium": {
                    "Female": (20, 50),
                    "Male": (10, 30)
                },
                "premium": {
                    "Female": (50, 200),
                    "Male": (30, 75)
                }
            }
        },
        "price_month_coefficients": [(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, 1.1), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 0.95), (12, 0.85)],
        "quantity": {
            "Female": [(1, 1), (2, 2), (3, 3), (4, 2), (5, 1), (6, 0.5), (7, 0.25), (8, 0.15), (9, 0.1), (10, 0.05)],
            "Male": [(1, 3), (2, 1.5), (3, 0.5), (4, 0.15), (5, 0.05)]
        },
        "quantity_month_coefficients": [(1, 1.25), (2, 0.75), (3, 1.15), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.25)]
    },
    {
        "category": "Food & Beverage",
        "price": {
            "price_segments": [("budget", 5), ("medium", 1), ("premium", 0.5)],
            "price_range": {
                "budget": {
                    "Female": (4, 12),
                    "Male": (2, 10)
                },
                "medium": {
                    "Female": (12, 25),
                    "Male": (10, 20)
                },
                "premium": {
                    "Female": (25, 100),
                    "Male": (20, 50)
                }
            }
        },
        "price_month_coefficients": [(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, 1.1), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 0.95), (12, 0.85)],
        "quantity": {
            "Female": [(1, 0.15), (2, 0.25), (3, 0.5), (4, 1), (5, 3), (6, 3), (7, 3), (8, 2), (9, 1), (10, 0.5), (11, 0.25), (12, 0.15), (13, 0.1), (14, 0.05), (15, 0.025)],
            "Male": [(1, 0.5), (2, 0.5), (3, 0.05), (4, 0.025), (5, 0.05), (6, 0.25), (7, 0.5), (8, 1), (9, 1), (10, 1), (11, 2), (12, 2), (13, 2), (14, 2), (15, 3), (16, 3), (17, 3), (18, 3), (19, 3), (20, 3), (21, 1), (22, 1), (23, 0.5), (24, 0.25), (25, 0.1)]
        },
        "quantity_month_coefficients": [(1, 1.1), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 0.9), (8, 0.9), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.15)]
    },
    {
        "category": "Shoes",
        "price": {
            "price_segments": [("budget", 5), ("medium", 2), ("premium", 1)],
            "price_range": {
                "budget": {
                    "Female": (15, 50),
                    "Male": (10, 30)
                },
                "medium": {
                    "Female": (50, 150),
                    "Male": (30, 100)
                },
                "premium": {
                    "Female": (150, 500),
                    "Male": (100, 300)
                }
            }
        },
        "price_month_coefficients": [(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, 1.1), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 0.95), (12, 0.85)],
        "quantity": {
            "Female": [(1, 1)],
            "Male": [(1, 1)]
        },
        "quantity_month_coefficients": [(1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0)]
    },
    {
        "category": "Souvenir",
        "price": {
            "price_segments": [("budget", 2), ("medium", 2), ("premium", 0.5)],
            "price_range": {
                "budget": {
                    "Female": (0.5, 1),
                    "Male": (0.5, 1)
                },
                "medium": {
                    "Female": (1, 3),
                    "Male": (1, 3)
                },
                "premium": {
                    "Female": (3, 10),
                    "Male": (3, 10)
                }
            }
        },
        "price_month_coefficients": [(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, 1.1), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 0.95), (12, 0.85)],
        "quantity": {
            "Female": [(1, 0.15), (2, 0.25), (3, 0.5), (4, 1), (5, 3), (6, 3), (7, 3), (8, 2), (9, 1), (10, 0.5)],
            "Male": [(1, 1), (2, 1), (3, 1), (4, 0.5), (5, 0.5), (6, 0.25), (7, 0.25), (8, 0.1), (9, 0.05), (10, 0.025)]
        },
        "quantity_month_coefficients": [(1, 1.25), (2, 0.75), (3, 1.15), (4, 1.0), (5, 1.0), (6, 1.15), (7, 1.5), (8, 2.0), (9, 1.15), (10, 1.0), (11, 1.0), (12, 1.25)]
    },
    {
        "category": "Technology",
        "price": {
            "price_segments": [("budget", 5), ("medium", 2), ("premium", 1)],
            "price_range": {
                "budget": {
                    "Female": (10, 50),
                    "Male": (10, 50)
                },
                "medium": {
                    "Female": (50, 500),
                    "Male": (50, 500)
                },
                "premium": {
                    "Female": (500, 2000),
                    "Male": (500, 2000)
                }
            }
        },
        "price_month_coefficients": [(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, 1.1), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 0.95), (12, 0.85)],
        "quantity": {
            "Female": [(1, 2), (2, 0.5), (3, 0.05)],
            "Male": [(1, 5), (2, 3), (3, 1), (4, 0.25), (5, 0.025)]
        },
        "quantity_month_coefficients": [(1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 1.15), (12, 1.25)]
    },
    {
        "category": "Toys",
        "price": {
            "price_segments": [("budget", 5), ("medium", 2), ("premium", 1)],
            "price_range": {
                "budget": {
                    "Female": (5, 10),
                    "Male": (5, 10)
                },
                "medium": {
                    "Female": (10, 20),
                    "Male": (10, 15)
                },
                "premium": {
                    "Female": (20, 50),
                    "Male": (15, 50)
                }
            }
        },
        "price_month_coefficients": [(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, 1.1), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 0.95), (12, 0.85)],
        "quantity": {
            "Female": [(1, 3), (2, 2), (3, 0.5)],
            "Male": [(1, 3), (2, 0.5), (3, 0.025)]
        },
        "quantity_month_coefficients": [(1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.25)]
    },
]

config_df = pd.DataFrame(config_dict)
config_df.set_index("category", inplace=True)

config_df

,price,price_month_coefficients,quantity,quantity_month_coefficients
category,,,,
Books,"{'price_segments': [('budget', 5), ('medium', ...","[(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, ...","{'Female': [(1, 3), (2, 1), (3, 0.5)], 'Male':...","[(1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1..."
Clothing,"{'price_segments': [('budget', 5), ('medium', ...","[(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, ...","{'Female': [(1, 2), (2, 3), (3, 2), (4, 1), (5...","[(1, 1.25), (2, 0.75), (3, 1.15), (4, 1.0), (5..."
Cosmetics,"{'price_segments': [('budget', 5), ('medium', ...","[(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, ...","{'Female': [(1, 1), (2, 2), (3, 3), (4, 2), (5...","[(1, 1.25), (2, 0.75), (3, 1.15), (4, 1.0), (5..."
Food & Beverage,"{'price_segments': [('budget', 5), ('medium', ...","[(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, ...","{'Female': [(1, 0.15), (2, 0.25), (3, 0.5), (4...","[(1, 1.1), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1..."
Shoes,"{'price_segments': [('budget', 5), ('medium', ...","[(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, ...","{'Female': [(1, 1)], 'Male': [(1, 1)]}","[(1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1..."
Souvenir,"{'price_segments': [('budget', 2), ('medium', ...","[(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, ...","{'Female': [(1, 0.15), (2, 0.25), (3, 0.5), (4...","[(1, 1.25), (2, 0.75), (3, 1.15), (4, 1.0), (5..."
Technology,"{'price_segments': [('budget', 5), ('medium', ...","[(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, ...","{'Female': [(1, 2), (2, 0.5), (3, 0.05)], 'Mal...","[(1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1..."
Toys,"{'price_segments': [('budget', 5), ('medium', ...","[(1, 1.0), (2, 1.2), (3, 1.15), (4, 1.1), (5, ...","{'Female': [(1, 3), (2, 2), (3, 0.5)], 'Male':...","[(1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1..."


In [465]:
def calculate_random_price_quantity(category, gender, invoice_date, quantity):
    #convert date
    date = datetime.strptime(invoice_date, "%d/%m/%Y").strftime("%Y-%m-%d")
    month = datetime.strptime(invoice_date, "%d/%m/%Y").month

    #read configuration
    price_segment_weights_list = config_df.loc[category]["price"]["price_segments"]
    price_month_coefficient = config_df.loc[category]["price_month_coefficients"][month - 1][1]

    quantity_weights_list = config_df.loc[category]["quantity"][gender]
    quantity_base_month_coefficient = config_df.loc[category]["quantity_month_coefficients"][month - 1][1]

    #process configuration
    price_segment_list = [x[0] for x in price_segment_weights_list]
    price_segment_weights = [x[1] for x in price_segment_weights_list]

    price_segment = random.choices(price_segment_list, weights=price_segment_weights, k=1)[0]

    min_price = config_df.loc[category]["price"]["price_range"][price_segment][gender][0]
    max_price = config_df.loc[category]["price"]["price_range"][price_segment][gender][1]

    quantity_list = [x[0] for x in quantity_weights_list]
    quantity_weights = [x[1] for x in quantity_weights_list]

    #get exchange rate
    exchange_rate = exchange_rate_df.loc[date][0]

    #calculate random price
    exchange_rate_random_coefficient = random.uniform(0.9, 1.1)

    base_random_price = random.uniform(min_price, max_price)

    random_price = round(base_random_price * price_month_coefficient * exchange_rate * exchange_rate_random_coefficient, 2)

    #calculate random quantity
    #exeption is Shoes category, keep the original quantity
    if category != "Shoes":
        base_random_quantity = random.choices(quantity_list, weights=quantity_weights, k=1)[0]

        random_quantity = round(base_random_quantity * quantity_base_month_coefficient)

        if random_quantity == 0:
            random_quantity = 1
    else:
        random_quantity = quantity

    return random_price, random_quantity

In [466]:
customer_shopping_data_df[["random_price", "random_quantity"]] = \
    customer_shopping_data_df.apply(lambda row: pd.Series(calculate_random_price_quantity(category=row["category"], 
                                                                                          gender=row["gender"],
                                                                                          invoice_date=row["invoice_date"],
                                                                                          quantity=row["quantity"])), axis=1)

customer_shopping_data_df.head()


,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,random_price,random_quantity
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon,16107.36,4.0
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul,220.54,3.0
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity,180.43,1.0
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM,848.38,5.0
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon,10.18,1.0


In [467]:
customer_shopping_data_df["random_quantity"] = customer_shopping_data_df["random_quantity"].astype(int)
customer_shopping_data_df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,random_price,random_quantity
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon,16107.36,4
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul,220.54,3
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity,180.43,1
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM,848.38,5
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon,10.18,1


In [468]:
customer_shopping_data_df.drop(columns=["quantity", "price"], inplace=True)
customer_shopping_data_df.rename(columns={"random_price": "price", "random_quantity": "quantity"}, inplace=True)

customer_shopping_data_df = customer_shopping_data_df[["invoice_no", "customer_id", "gender", "age", "category", "quantity", "price", "payment_method", "invoice_date", "shopping_mall"]]

customer_shopping_data_df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,4,16107.36,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,220.54,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,180.43,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,848.38,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,1,10.18,Cash,24/10/2021,Kanyon


In [469]:
customer_shopping_data_df.to_csv("./Resources/source/customer_shopping_data.csv", header=True, index=False)